# Assignment 1 Workbook
## Nguyen Vu

   

In [ ]:
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q http://mirror.cc.columbia.edu/pub/software/apache/spark/spark-3.0.0-preview2/spark-3.0.0-preview2-bin-hadoop2.7.tgz
# !tar xf spark-3.0.0-preview2-bin-hadoop2.7.tgz
# !pip install -q findspark
# !rm -rf spark-3.0.0-preview2-bin-hadoop2.7.tgz

In [3]:
import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.0.0-preview2-bin-hadoop2.7"

#import findspark
# findspark.init("spark-3.0.0-preview2-bin-hadoop2.7")# SPARK_HOME

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()

In [4]:
ls -lah

total 104
drwxr-xr-x  11 nguyendvu  staff   352B Jul 16 19:21 ./
drwxr-xr-x@ 82 nguyendvu  staff   2.6K Jul 16 14:29 ../
-rw-r--r--@  1 nguyendvu  staff    10K Jul 16 14:35 .DS_Store
drwxr-xr-x   4 nguyendvu  staff   128B Jul 15 18:59 .ipynb_checkpoints/
-rw-r--r--@  1 nguyendvu  staff    26K Jul 16 19:21 Assignment1.ipynb
-rw-r--r--   1 nguyendvu  staff   8.7K Jul 15 19:39 Assignment1_OLD.ipynb
drwxr-xr-x   5 nguyendvu  staff   160B Jul 11 12:39 Assignment1_dt/
drwxr-xr-x   9 nguyendvu  staff   288B Jul 16 14:34 BigDataNBs/
drwxr-xr-x   6 nguyendvu  staff   192B Jul 15 21:34 top10Drivers/
drwxr-xr-x   6 nguyendvu  staff   192B Jul 15 21:34 top10Taxi/
drwxr-xr-x   6 nguyendvu  staff   192B Jul 15 21:37 top10WorkHours/


In [5]:
!ls -la

total 104
drwxr-xr-x  11 nguyendvu  staff    352 Jul 16 19:21 .
drwxr-xr-x@ 82 nguyendvu  staff   2624 Jul 16 14:29 ..
-rw-r--r--@  1 nguyendvu  staff  10244 Jul 16 14:35 .DS_Store
drwxr-xr-x   4 nguyendvu  staff    128 Jul 15 18:59 .ipynb_checkpoints
-rw-r--r--@  1 nguyendvu  staff  26566 Jul 16 19:21 Assignment1.ipynb
-rw-r--r--   1 nguyendvu  staff   8916 Jul 15 19:39 Assignment1_OLD.ipynb
drwxr-xr-x   5 nguyendvu  staff    160 Jul 11 12:39 Assignment1_dt
drwxr-xr-x   9 nguyendvu  staff    288 Jul 16 14:34 BigDataNBs
drwxr-xr-x   6 nguyendvu  staff    192 Jul 15 21:34 top10Drivers
drwxr-xr-x   6 nguyendvu  staff    192 Jul 15 21:34 top10Taxi
drwxr-xr-x   6 nguyendvu  staff    192 Jul 15 21:37 top10WorkHours


In [6]:
!install conda install -c anaconda wget
!wget https://s3.amazonaws.com/metcs777/taxi-data-sorted-small.csv.bz2

usage: install [-bCcpSsv] [-B suffix] [-f flags] [-g group] [-m mode]
               [-o owner] file1 file2
       install [-bCcpSsv] [-B suffix] [-f flags] [-g group] [-m mode]
               [-o owner] file1 ... fileN directory
       install -d [-v] [-g group] [-m mode] [-o owner] directory ...
/bin/sh: wget: command not found


In [7]:
#check if spark are running

print(sc.version)


3.0.0


In [8]:
lines = sc.textFile("Assignment1_dt/taxi-data-sorted-small.csv.bz2")
taxilines = lines.map(lambda line: line.split(","))

In [9]:
#print first row of the data to make sure it is read in correctly
print(taxilines.take(2))

[['07290D3599E7A0D62097A346EFCC1FB5', 'E7750A37CAB07D0DFF0AF7E3573AC141', '2013-01-01 00:00:00', '2013-01-01 00:02:00', '120', '0.44', '-73.956528', '40.716976', '-73.962440', '40.715008', 'CSH', '3.50', '0.50', '0.50', '0.00', '0.00', '4.50'], ['22D70BF00EEB0ADC83BA8177BB861991', '3FF2709163DE7036FCAA4E5A3324E4BF', '2013-01-01 00:02:00', '2013-01-01 00:02:00', '0', '0.00', '0.000000', '0.000000', '0.000000', '0.000000', 'CSH', '27.00', '0.00', '0.50', '0.00', '0.00', '27.50']]


Use pycode from assignment 1 document to clean up the data loaded

In [10]:
#clean up 
#handling wrong data lines
def isfloat(val):
    try: 
        float(val)
        return True
    except:
        return False
    
#use isFloat to remove lines if they dont have 16 values and a float value !=0 for col 6 and col 12
def correctRows(p):
    if(len(p)==17):
        if(isfloat(p[5]) and isfloat(p[11])):
            if(float(p[5])!=0 and float(p[11])!=0):
                return p
            
#go thru each element per row and keep only the correctRows
cleanTaxiDT = taxilines.filter(correctRows)

### Part1: Top-10 Active Taxis
Many different taxis have had multiple drivers. Write and execute a Spark Python program that computes
the top ten taxis that have had the largest number of drivers. Your output should be a set of (medallion,
number of drivers) pairs


In [11]:
#set up each row with an 1 index
rdd_1 = cleanTaxiDT.map(lambda x: ((x[0], x[1]), 1))

In [12]:
rdd_1.first()

(('07290D3599E7A0D62097A346EFCC1FB5', 'E7750A37CAB07D0DFF0AF7E3573AC141'), 1)

In [13]:
#reduce, aggrgate by the combination of taxi and taxi drivers into a list
rdd_1_reduce = rdd_1.reduceByKey(lambda x, y: x+y)


In [14]:
# reduce in RDD to count taxi and most frequently used taxi
by_medalionRDD2 = rdd_1_reduce.map(lambda x: (x[0][0],1)).reduceByKey(lambda x, y: x+y)

In [ ]:
#return the top 10 most used taxi by the count
top10taxi = sc.parallelize(by_medalionRDD2.top(10, lambda x: x[1]))

top10taxi.coalesce(1).saveAsTextFile("top10Taxi/")

### Part2: Top-10 Best Drivers
We would like to figure out who the top 10 best drivers are in terms of their average earned money per minute spent carrying a customer. The total amount field is the total money earned on a trip. In the end, we
are interested in computing a set of (driver, money per minute) pairs

Create a second clean RDD to remove any rows that contain no value for the minute column and save to cleanTaxiDT2

In [16]:
def correctRows2(p):
  if(float(p[4]) !=0):
    return p
cleanTaxiDT2 = cleanTaxiDT.filter(correctRows2)

In [ ]:
# check to see if the cleaned up data contail any zero value for column 5
cleanTaxiDT2.filter(lambda x: float(x[4])==0 ).count()

In [17]:
#get a driver rdd and map a function to get the money per minute value
by_drvr = cleanTaxiDT2.map(lambda x: (x[1], (float(x[11])/(float(x[4])/60), 1.0)))
#aggregate by key=driverID and tke an average using the aggregate byByKey
aggfun = lambda x, y: (x[0]+y[0], x[1]+y[1])
rateRDD = by_drvr.aggregateByKey((0.0,0.0), aggfun, aggfun)

avg_rateRDD = rateRDD.map(lambda x: (x[0], x[1][0]/x[1][1]))


In [ ]:
# Get top 10 drivers
top10drivers = sc.parallelize(avg_rateRDD.top(10, lambda x: x[1]))
top10drivers.coalesce(1).saveAsTextFile("top10Drivers/")

### Part3: Best Day to Work
We would like to know which hour of the day is the best time for drivers that has the highest profit per miles. Consider the surcharge amount in dollar for each taxi ride (without tip amount) and the distance in miles, and sum up the rides for each hour of the day (24 hours) – consider the pickup time for your calculation. The profit ratio is the ration surcharge in dollar divided by the travel distance in miles for each specific time
of the day. 

Profit Ratio = (Surcharge Amount in US Dollar) / (Travel Distance in miles)
We are interested to know the time of the day that has the highest profit ratio.

In [19]:
#col index: surcharge=12, Distance=5, pickup datetime=2

# get the day out of the datetime column
from datetime import datetime

In [20]:

# create a method to extract hour
def getHour(x):
  dtObj = datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
  return dtObj.strftime('%H')

#create aRDD with the relevant columns: pick up date, surcharge, distance
pt3RDD = cleanTaxiDT.map(lambda x: (getHour(x[2]), (float(x[12]), float(x[5]))))
#RDD to contain sum of all surcharge, all distance for each date as key
byHourRDD = pt3RDD.reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1]))
#RDD to produce date and profit ratio by day
bestDayRDD = byHourRDD.map(lambda x: (x[0], x[1][0]/x[1][1]))



In [71]:
bestDayRDD.first()

('04', 0.11624931403920391)

In [ ]:
bestWrkHr = sc.parallelize(bestDayRDD.top(10, lambda x: x[1]))

#bestWrkHr.coalesce(1).saveAsTextFile("top10WorkHours/")

### Part4: Bonus 
How many percent of taxi customers pay with cash and how many percent using electronic cards?
Analyze these payment methods for different time of the day and provide a list of percents for each
day time? 

As a result provide two numbers for total percentages and a list like (hour of day, percent paid card) We would like to measure the efficiency of taxis drivers by finding out their average earned money per mile. (Consider the total amount which includes tips, as their earned money) Implement a Spark job
that can find out the top-10 efficient taxi divers.

What are mean, median, first and third quantiles of tip amount? How do find the median?
Using the IQR outlier detection method find out the top-10 outliers.

In [21]:
# cash versus credit
# columns: pickup date = 2, payment type = 10, trip distance = 5, fare amount = 11, surcharge = 12, tax = 13, tip = 14
from pyspark.sql.types import *
import pyspark.sql.functions as F

rdd_transf = cleanTaxiDT.map(lambda x: [x[num] for num in [0, 1, 2, 10, 5, 11, 12, 13, 14]])

colnames = ['TaxiID','DriverID','DateTime', 'PaymentType', 'Distance', 'Fare', 'Surcharge', 'Tax', 'Tip']


sFields = [StructField(field, StringType(), True) for field in colnames]

schema = StructType(sFields)

df_bonus = sqlContext.createDataFrame(rdd_transf, schema)

rename_col = colnames[4:9]


In [22]:
# corrected DF to cask certain columns but this will change the DF
correctedDF = df_bonus.select(*(F.col(c).cast('float') for c in rename_col))

In [23]:
#rename all columns and maintain the other columns.
for colname in rename_col:
    df_bonus = df_bonus.withColumn(colname, F.col(colname).cast('float'))
    
#add column for date and hour
# create a method to extract hour
def getHour(x):
  dtObj = datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
  return dtObj.strftime('%H')

#establish UDF
udfGetHour = F.udf(getHour, StringType())

df_bonus = df_bonus.withColumn('Hour', udfGetHour(df_bonus.DateTime))

In [ ]:
df_bonus.dtypes

In [79]:
df_bonus.show(1)

+--------------------+--------------------+-------------------+-----------+--------+----+---------+---+---+----+
|              TaxiID|            DriverID|           DateTime|PaymentType|Distance|Fare|Surcharge|Tax|Tip|Hour|
+--------------------+--------------------+-------------------+-----------+--------+----+---------+---+---+----+
|07290D3599E7A0D62...|E7750A37CAB07D0DF...|2013-01-01 00:00:00|        CSH|    0.44| 3.5|      0.5|0.5|0.0|  00|
+--------------------+--------------------+-------------------+-----------+--------+----+---------+---+---+----+
only showing top 1 row



In [44]:
#get the count split up by payment type as a total of all payment 
paymentType = df_bonus.groupBy('PaymentType').count()

total = df_bonus.count()

#add % by total column to calculate count by total rows
paymentTypeOut = paymentType.withColumn("%_of_total", (F.col('count')/total)*100)
#total %paid by different methods
paymentTypeOut.rdd.map(list).coalesce(1).saveAsTextFile("percentCardVsCash/")




In [45]:
#analyze average earn per mile

df_bonus = df_bonus.withColumn('dollarPerMile', 
                               ((F.col('Fare')+F.col('Surcharge')+F.col('Tax')+F.col('Tip'))/F.col('Distance')))

df_bonus.show(1)

bydriverdf = df_bonus.groupBy('DriverID').mean('dollarPerMile')

top10DriverByMile = bydriverdf.orderBy("avg(dollarPerMile)", ascending=0).head(10)

top10DriverByMileRDD = sc.parallelize(top10DriverByMile)

top10DriverByMileRDD.coalesce(1).saveAsTextFile("top10DriverByMile/")

+--------------------+--------------------+-------------------+-----------+--------+----+---------+---+---+----+-----------------+
|              TaxiID|            DriverID|           DateTime|PaymentType|Distance|Fare|Surcharge|Tax|Tip|Hour|    dollarPerMile|
+--------------------+--------------------+-------------------+-----------+--------+----+---------+---+---+----+-----------------+
|07290D3599E7A0D62...|E7750A37CAB07D0DF...|2013-01-01 00:00:00|        CSH|    0.44| 3.5|      0.5|0.5|0.0|  00|10.22727278269027|
+--------------------+--------------------+-------------------+-----------+--------+----+---------+---+---+----+-----------------+
only showing top 1 row



In [ ]:
#get the summary of statistics for Tips, and quantiles of tips amount
df_bonus.describe('Tip').rdd.map(list).coalesce(1).saveAsTextFile("TipsSummaryStats/")

tipsQuantiles = df_bonus.approxQuantile('Tip', [0.25, 0.5, 0.75], 0.25)

sc.parallelize(tipsQuantiles).coalesce(1).saveAsTextFile("TipsQuantiles/")
